# sklearn数据集训练逻辑回归模型

In [293]:
import numpy as np
from sklearn.model_selection import train_test_split

In [294]:
# 模型计算函数
def forward(x, theta, bias):
    # 线性运算
    z = np.dot(theta, x.T) + bias # shape (105,10)
    # sigmoid
    y_hat = 1 / (1 + np.exp(-z))  # shape (105,10)
    return y_hat

# 计算损失函数
def loss(y, y_hat):
    e = 1e-8
    return - y * np.log(y_hat + e) - (1 - y) * np.log(1 - y_hat + e)

# 计算梯度
def calc_gradient(x,y,y_hat):
    # 计算梯度
    m = x.shape[-1]
    # theta梯度计算
    delta_theta = np.dot((y_hat - y), x) / m
    # bias梯度计算
    delta_bias = np.mean(y_hat - y)
    # 返回梯度
    return delta_theta, delta_bias

In [295]:
def train_model(X_train, y_train, theta, bias = 0, lr=0.1, epochs = 3000):
    for i in range(epochs):
        # 前向计算
        y_hat = forward(X_train, theta, bias)
        # 计算损失
        loss_val = loss(y_train, y_hat)
        # 计算梯度
        delta_theta, delta_bias = calc_gradient(X_train, y_train, y_hat)
        # 更新参数
        theta = theta - lr * delta_theta
        bias = bias - lr * delta_bias

        if i % 100 == 0:
            # 计算准确率
            acc = np.mean(np.round(y_hat) == y_train)  # [False,True,...,False] -> [0,1,...,0]
            print(f"epoch: {i}, loss: {np.mean(loss_val)}, acc: {acc}")

### make_classification数据集

In [296]:
from sklearn.datasets import make_classification
# 生成训练数据
X,y = make_classification(n_samples=150, n_features=10)  # shape (150, 10)
print(f"make_classification X:\n{X}")
print(f"make_classification y:\n{y}")

# 拆分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# 权重参数
theta = np.random.randn(1,10)  # shape (1, 10)
# 偏置
bias = 0
# 超参数
lr = 0.1
epochs = 3000  # 训练次数

train_model(X_train,y_train,theta,bias,lr,epochs)

make_classification X:
[[-0.10327224 -0.13476427 -0.26925331 ...  0.4271644  -1.39921354
   0.43196846]
 [-0.06765103  0.3100076   0.87836899 ... -1.1094204   1.50068729
  -0.23662867]
 [-0.69315251  1.09337805  0.63320277 ...  2.14591683  0.57675544
  -0.41291847]
 ...
 [ 0.27980246 -0.75550505  0.10772763 ...  0.08435208  1.49905772
  -0.4845368 ]
 [-0.09416606  0.83872564 -0.07830347 ... -0.80212615  1.83587774
  -0.57259002]
 [-1.00149727 -0.24938999 -0.40672114 ... -0.75446975 -0.29441958
  -0.16529389]]
make_classification y:
[0 0 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 0 1 1 0 1 0 0 0 1 0 1 0 1 0 0 1 1 1 0
 1 1 0 1 0 0 1 0 1 0 1 0 0 0 0 0 1 0 1 0 0 1 1 0 1 0 1 1 1 1 1 0 0 1 1 0 0
 0 0 0 1 0 0 0 1 0 1 1 1 0 1 0 1 1 1 0 0 1 0 1 1 0 0 0 1 0 0 1 0 1 1 1 1 1
 0 1 0 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 1 1 1 0 0 1 1 1 1 0 0 0 0 0 1
 0 1]
epoch: 0, loss: 2.8275370670591053, acc: 0.41904761904761906
epoch: 100, loss: 0.28467667680177755, acc: 0.8571428571428571
epoch: 200, loss: 0.280532568722

In [297]:
# 模型推理
idx = np.random.randint(len(X_test)) # 随机选择一个测试样本索引
x = X_test[idx]
y = y_test[idx]

predict = np.round(forward(x, theta, bias))
print(f"y: {y}, predict: {predict}")

y: 1, predict: [1.]


### load_iris鸢尾花数据集

In [298]:
# 修正前向传播函数（对齐形状）
def forward(x, theta, bias):
    z = np.dot(theta, x.T) + bias   # Shape: (1, n_samples)
    y_hat = 1 / (1 + np.exp(-z))    # Shape: (1, n_samples)
    return y_hat.reshape(-1)        # 输出形状转为 (n_samples, )

# 修正损失函数和梯度计算（确保形状一致）
def loss(y, y_hat):
    e = 1e-8
    return -y * np.log(y_hat + e) - (1 - y) * np.log(1 - y_hat + e)

def calc_gradient(x, y, y_hat):
    m = x.shape[0]
    error = (y_hat - y).reshape(-1, 1)  # 形状调整为 (n_samples, 1)
    delta_theta = np.dot(error.T, x) / m  # (1, 4)
    delta_bias = np.mean(error)          # 标量
    return delta_theta, delta_bias

In [299]:
from sklearn.datasets import load_iris

# 1. 加载并预处理数据
X,y = load_iris(return_X_y=True)
X=X[:100]
y=y[:100]

# iris=load_iris()
# X = iris.data[iris.target < 2, :]  # iris有4个特征数为 4
# y = iris.target[iris.target < 2]   # 目标值为 0 和 1

# 拆分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42) 
# stratify=y 保持训练集和测试集中的类别分布比例与原始数据集 y 一致
# random_state=42，控制数据分割过程的“随机性”，确保每次运行时随机划分的结果完全一致

# 2. 调整参数初始化
theta = np.random.randn(1, 4)  # 特征数为 4
bias = 0
lr = 0.1
epochs = 3000

np.save("theta.npy", theta)  # 保存文件 theta.npy
np.save("bias.npy", bias)    # 保存文件 bias.npy

train_model(X_train,y_train,theta,bias,lr,epochs)

epoch: 0, loss: 3.91434488872133, acc: 0.014285714285714285
epoch: 100, loss: 0.08776094814856074, acc: 1.0
epoch: 200, loss: 0.04046553852331824, acc: 1.0
epoch: 300, loss: 0.02666430892649154, acc: 1.0
epoch: 400, loss: 0.02003794659370268, acc: 1.0
epoch: 500, loss: 0.016126625932843596, acc: 1.0
epoch: 600, loss: 0.013536609208505548, acc: 1.0
epoch: 700, loss: 0.01169055728291672, acc: 1.0
epoch: 800, loss: 0.010305595353589908, acc: 1.0
epoch: 900, loss: 0.009226579026284027, acc: 1.0
epoch: 1000, loss: 0.008361201570942174, acc: 1.0
epoch: 1100, loss: 0.007651047720899814, acc: 1.0
epoch: 1200, loss: 0.007057314856974787, acc: 1.0
epoch: 1300, loss: 0.006553200609101172, acc: 1.0
epoch: 1400, loss: 0.006119585282063077, acc: 1.0
epoch: 1500, loss: 0.005742455930860833, acc: 1.0
epoch: 1600, loss: 0.005411303328500331, acc: 1.0
epoch: 1700, loss: 0.005118088101557182, acc: 1.0
epoch: 1800, loss: 0.004856553188598652, acc: 1.0
epoch: 1900, loss: 0.0046217542801054515, acc: 1.0
epo

In [311]:
# lr=0.01
train_model(X_train,y_train,theta,bias,0.01,epochs)

epoch: 0, loss: 3.91434488872133, acc: 0.014285714285714285
epoch: 100, loss: 1.8963059636344788, acc: 0.0
epoch: 200, loss: 0.9374595083424571, acc: 0.15714285714285714
epoch: 300, loss: 0.4965939625456336, acc: 0.9857142857142858
epoch: 400, loss: 0.31232775553063646, acc: 1.0
epoch: 500, loss: 0.22258030055892974, acc: 1.0
epoch: 600, loss: 0.17161676765103467, acc: 1.0
epoch: 700, loss: 0.13929354486321405, acc: 1.0
epoch: 800, loss: 0.11712909011365663, acc: 1.0
epoch: 900, loss: 0.10104419386152935, acc: 1.0
epoch: 1000, loss: 0.08886324053695117, acc: 1.0
epoch: 1100, loss: 0.07932919389236259, acc: 1.0
epoch: 1200, loss: 0.0716685371198094, acc: 1.0
epoch: 1300, loss: 0.06538060233468627, acc: 1.0
epoch: 1400, loss: 0.06012764807008512, acc: 1.0
epoch: 1500, loss: 0.055673874348804185, acc: 1.0
epoch: 1600, loss: 0.05184981277843653, acc: 1.0
epoch: 1700, loss: 0.04853061572602036, acc: 1.0
epoch: 1800, loss: 0.04562232185946802, acc: 1.0
epoch: 1900, loss: 0.043052886156085005

In [332]:
# bias=1
train_model(X_train,y_train,theta,1,lr,epochs)

epoch: 0, loss: 3.8075852182457597, acc: 0.0
epoch: 100, loss: 0.0888982569055974, acc: 1.0
epoch: 200, loss: 0.04077312638168462, acc: 1.0
epoch: 300, loss: 0.026811546210199405, acc: 1.0
epoch: 400, loss: 0.02012483793482428, acc: 1.0
epoch: 500, loss: 0.016183817727171916, acc: 1.0
epoch: 600, loss: 0.013576860461629824, acc: 1.0
epoch: 700, loss: 0.011720195502899601, acc: 1.0
epoch: 800, loss: 0.010328138155749744, acc: 1.0
epoch: 900, loss: 0.009244146365011127, acc: 1.0
epoch: 1000, loss: 0.008375150359363934, acc: 1.0
epoch: 1100, loss: 0.007662287965777706, acc: 1.0
epoch: 1200, loss: 0.007066480010241575, acc: 1.0
epoch: 1300, loss: 0.006560745234804734, acc: 1.0
epoch: 1400, loss: 0.006125843937360069, acc: 1.0
epoch: 1500, loss: 0.005747680171611094, acc: 1.0
epoch: 1600, loss: 0.005415685795411093, acc: 1.0
epoch: 1700, loss: 0.005121778673105254, acc: 1.0
epoch: 1800, loss: 0.004859670097137411, acc: 1.0
epoch: 1900, loss: 0.004624391936159539, acc: 1.0
epoch: 2000, loss:

In [300]:
# 模型推理
# 随机选择一个测试样本索引
random_index = np.random.choice(X_test.shape[0])  # X_test形状为 (n_samples, 4)
sample_x = X_test[random_index]                   # 特征向量，shape (4,)
true_label = y_test[random_index]                 # 真实标签，0或1
# 调整输入形状（注意模型接收的是 (1,4)，而非 (4,)）
sample_x_reshaped = sample_x.reshape(1, -1)  # 转换为 (1, 4)
# 调用前向传播函数
y_hat_single = forward(sample_x_reshaped, theta, bias)  # 输出形状 (1,)
predicted_prob = y_hat_single[0]                        # 提取标量概率值
predicted_class = 1 if predicted_prob >= 0.5 else 0     # 根据阈值分类
print(f"随机选择的测试样本索引: {random_index}")
print(f"特征值: {sample_x}")
print(f"预测概率: {predicted_prob:.4f} → 预测类别: {predicted_class}")
print(f"真实类别: {true_label}")
print("预测结果是否正确:", "是" if predicted_class == true_label else "否")


随机选择的测试样本索引: 2
特征值: [5.4 3.  4.5 1.5]
预测概率: 0.0014 → 预测类别: 0
真实类别: 1
预测结果是否正确: 否


In [301]:
theta = np.load("theta.npy")  # 形状 (1, 4)
bias = np.load("bias.npy")    # 标量值
train_model(X_train,y_train,theta,bias,lr,epochs)

epoch: 0, loss: 3.91434488872133, acc: 0.014285714285714285
epoch: 100, loss: 0.08776094814856074, acc: 1.0
epoch: 200, loss: 0.04046553852331824, acc: 1.0
epoch: 300, loss: 0.02666430892649154, acc: 1.0
epoch: 400, loss: 0.02003794659370268, acc: 1.0
epoch: 500, loss: 0.016126625932843596, acc: 1.0
epoch: 600, loss: 0.013536609208505548, acc: 1.0
epoch: 700, loss: 0.01169055728291672, acc: 1.0
epoch: 800, loss: 0.010305595353589908, acc: 1.0
epoch: 900, loss: 0.009226579026284027, acc: 1.0
epoch: 1000, loss: 0.008361201570942174, acc: 1.0
epoch: 1100, loss: 0.007651047720899814, acc: 1.0
epoch: 1200, loss: 0.007057314856974787, acc: 1.0
epoch: 1300, loss: 0.006553200609101172, acc: 1.0
epoch: 1400, loss: 0.006119585282063077, acc: 1.0
epoch: 1500, loss: 0.005742455930860833, acc: 1.0
epoch: 1600, loss: 0.005411303328500331, acc: 1.0
epoch: 1700, loss: 0.005118088101557182, acc: 1.0
epoch: 1800, loss: 0.004856553188598652, acc: 1.0
epoch: 1900, loss: 0.0046217542801054515, acc: 1.0
epo

逻辑回归运算及训练涉及多个关键知识点，具体如下：
1. **逻辑回归模型**：将线性回归模型输出从实数空间映射到[0, 1]区间，转化为概率值，通过sigmoid函数实现，即 $ \hat{y} $ = sigmoid(z) =$ \frac{1}{1 + e^{-z}} $。
2. **最大似然估计与损失函数**
    - **最大似然估计**：用于生成拟合数据分布的最可能参数估计，目的是找到符合当前观测数据的概率分布。
    - **似然函数与对数似然函数**：假设数据服从某分布，似然函数通过观测值计算数据在假设分布中的出现可能性，对数似然函数对似然函数取对数，将乘法运算变为加法运算，便于计算。
    - **损失函数**：逻辑回归中，引入假设函数预测 $Y$ 为1和0的概率，基于最大似然估计和伯努利分布得到似然函数，取对数并添加负号后得到损失函数 $ J \theta =-\sum_{i}^{m} Y log \hat{Y}-(1 - Y) log (1-\hat{Y}) $，最小化该函数可得到理想的 $ \theta $值。
3. **梯度下降法**
    - **基本思想**：基于搜索的最优化方法，用于优化目标函数，在机器学习中用于最小化损失函数。其思想类比下山过程，通过寻找当前位置最陡峭方向（梯度反方向）向下走，逐步逼近损失函数最小值。
    - **梯度与学习率**：梯度是函数在某点的向量，方向与最大方向导数一致，模为方向导数最大值，在多元函数中，梯度方向指向函数上升最快方向，其反方向是下降最快方向，通过对权重参数求偏导数确定其影响。学习率是梯度下降中的超参数，影响梯度更新步长，合适的学习率可使损失函数在合适时间收敛到局部最小值，学习率过小收敛缓慢甚至不收敛，过大则会导致梯度在最小值附近震荡或无法收敛。
    - **模拟与可视化**：通过代码模拟梯度下降过程，可直观观察学习率对参数更新和收敛的影响，学习率的最佳取值与损失函数相关，常先设为0.01再寻找最佳值 。
4. **逻辑回归模型构建及训练流程（以numpy实现为例）**
    - **数据准备与参数初始化**：使用scikit-learn生成数据集并拆分为训练集和测试集，防止数据泄露，初始化权重参数 $ \theta $和偏置 $ bias $，设置学习率 $ Lr $ 和训练轮数 $ epoch $。
    - **模型运算（前向传播）**：定义前向运算函数，将自变量和权重参数代入模型，通过线性计算和sigmoid函数得到预测值 **$ \hat{y} $**。
    - **计算损失**：使用负对数损失函数计算损失，添加极小值防止 **$ \hat{y} $** 为0时出现错误。
    - **计算梯度**：根据公式计算权重参数和偏置的梯度。
    - **更新参数**：按照梯度下降公式更新参数，重复上述步骤，观察损失函数值调整学习率。